# Core

> Core functionality for checking docments compliance

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Any, Optional, Tuple, Callable
import re
import ast
import importlib
from dataclasses import dataclass
from fastcore.docments import docments, docstring

In [ ]:
#| export
@dataclass
class DocmentsCheckResult:    
    "Result of checking a function/class for docments compliance"
    name: str  # Name of the function/class
    type: str  # Type (FunctionDef, ClassDef, etc.)
    notebook: str  # Source notebook
    has_docstring: bool  # Whether it has a docstring
    params_documented: Dict[str, bool]  # Which params have documentation
    return_documented: bool  # Whether return is documented
    missing_params: List[str]  # Parameters missing documentation
    is_compliant: bool  # Overall compliance status
    source: str  # Source code of the definition
    has_todos: bool = False  # Whether it contains TODO placeholders
    todo_count: int = 0  # Number of TODO placeholders found
    params_with_type_hints: Dict[str, bool] = None  # Which params have type hints
    return_has_type_hint: bool = False  # Whether return has type hint
    params_missing_type_hints: List[str] = None  # Parameters missing type hints
    
    def __post_init__(self):
        "TODO: Add function description"
        if self.params_with_type_hints is None:
            self.params_with_type_hints = {}
        if self.params_missing_type_hints is None:
            self.params_missing_type_hints = []

In [ ]:
#| export
def extract_param_docs_from_func(
    func: Callable    # Function object to extract docs from
) -> Dict[str, str]:  # Mapping of parameter names to their documentation
    "Extract parameter documentation from function object using fastcore.docments"
    try:
        return docments(func)
    except Exception:
        # Fallback to original implementation if fastcore.docments fails
        import inspect
        source = inspect.getsource(func)
        return extract_param_docs(source)

In [ ]:
#| export
def extract_param_docs(
    source:str    # Function source code
) -> Dict[str, str]:  # Mapping of parameter names to their documentation
    "Extract parameter documentation from function source using docments style (fallback)"
    param_docs = {}
    lines = source.split('\n')
    in_params = False
    
    for i, line in enumerate(lines):
        stripped = line.strip()
        
        # Start looking after def line
        if stripped.startswith(('def ', 'async def ')):
            in_params = True
            continue
            
        # Stop when we hit the docstring
        if in_params and stripped.startswith(('"', "'", '"""', "'''")):
            break
            
        # Skip if not in parameters section
        if not in_params:
            continue
            
        # Match parameter with inline comment
        # Flexible regex that handles complex type annotations like Dict[str, Any]
        param_match = re.match(r'\s*(\w+)(?:[^#]*?)#\s*(.+)', line)
        if param_match:
            param_name = param_match.group(1)
            param_doc = param_match.group(2).strip()
            param_docs[param_name] = param_doc
    
    return param_docs

In [ ]:
#| export
def check_return_doc(
    source: str  # Function source code
) -> bool:  # Whether return is documented
    "Check if function has return documentation"
    lines = source.split('\n')
    
    for line in lines:
        # Check for return type annotation with comment
        if re.search(r'->\s*[^:]+\s*:\s*#\s*.+', line):
            return True
        # Check for standalone return comment
        if re.match(r'\s*->\s*[^:]+\s*$', line):
            # Check if next line has a comment
            idx = lines.index(line)
            if idx + 1 < len(lines):
                next_line = lines[idx + 1]
                if re.match(r'\s*#\s*.+', next_line):
                    return True
    
    return False

In [ ]:
#| export
def count_todos_in_docs(
    source: str,  # Function/class source code
    name: str  # Name of the function/class for AST parsing
) -> Tuple[int, bool]:  # (todo_count, has_todos)
    "Count TODO placeholders only in documentation (docstring, param docs, return docs)"
    todo_count = 0
    
    # Count TODOs in parameter comments
    param_docs = extract_param_docs(source)
    for doc in param_docs.values():
        todo_count += doc.count('TODO:')
    
    # Count TODOs in return type comments
    lines = source.split('\n')
    for line in lines:
        # Check for return type comment - must be return annotation followed by comment
        return_comment_match = re.search(r'\)\s*->\s*[^:#]+\s*:\s*#\s*(.*)', line)
        if return_comment_match:
            comment_text = return_comment_match.group(1)
            todo_count += comment_text.count('TODO:')
    
    # Count TODOs in docstring only
    try:
        tree = ast.parse(source)
        for node in ast.walk(tree):
            if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef, ast.ClassDef)):
                if node.name == name and node.body:
                    # Check if first statement is a docstring
                    first_stmt = node.body[0]
                    if (isinstance(first_stmt, ast.Expr) and 
                        isinstance(first_stmt.value, (ast.Str, ast.Constant))):
                        if hasattr(first_stmt.value, 's'):
                            docstring = first_stmt.value.s
                        elif hasattr(first_stmt.value, 'value'):
                            docstring = first_stmt.value.value
                        else:
                            docstring = ""
                        
                        if isinstance(docstring, str):
                            todo_count += docstring.count('TODO:')
                    break
    except:
        pass
    
    has_todos = todo_count > 0
    return todo_count, has_todos

In [ ]:
#| export
def check_has_docstring_from_func(
    func: Callable  # Function object to check
) -> bool:  # Whether the function has a docstring
    "Check if a function has a docstring using fastcore.docments"
    try:
        return docstring(func) is not None
    except Exception:
        # Fallback to AST parsing
        import inspect
        source = inspect.getsource(func)
        return check_has_docstring(source, func.__name__)

In [ ]:
#| export
def check_has_docstring(
    source: str,  # Function/class source code
    name: str  # Name of the function/class
) -> bool:  # Whether the definition has a docstring
    "Check if a function/class has a docstring using AST parsing (fallback)"
    has_docstring = False
    try:
        tree = ast.parse(source)
        for node in ast.walk(tree):
            if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef, ast.ClassDef)):
                if node.name == name:
                    # Check if first statement in body is a string (docstring)
                    if (node.body and 
                        isinstance(node.body[0], ast.Expr) and 
                        isinstance(node.body[0].value, (ast.Str, ast.Constant))):
                        has_docstring = True
                    break
    except:
        # Fallback to simple string detection
        has_docstring = any(line.strip().startswith(('"', "'")) for line in source.split('\n')[1:5])
    
    return has_docstring

In [ ]:
#| export
def check_type_hints(
    definition: Dict[str, Any]  # Definition dict from scanner
) -> Tuple[Dict[str, bool], List[str], bool]:  # (params_with_type_hints, missing_type_hints, return_has_type_hint)
    "Check which parameters and return value have type hints"
    params_with_type_hints = {}
    missing_type_hints = []
    return_has_type_hint = False
    
    # Only check for functions
    if definition['type'] in ['FunctionDef', 'AsyncFunctionDef']:
        # Check each parameter for type hints
        for arg in definition.get('args', []):
            param_name = arg['name']
            if param_name != 'self':  # Skip self parameter
                has_type_hint = arg.get('annotation') is not None
                params_with_type_hints[param_name] = has_type_hint
                if not has_type_hint:
                    missing_type_hints.append(param_name)
        
        # Check return type hint, but skip for special methods that conventionally don't need return annotations
        function_name = definition.get('name', '')
        special_methods = ['__init__', '__post_init__', '__enter__', '__exit__', '__del__', '__setattr__', '__delattr__', '__setitem__', '__delitem__']
        
        if function_name not in special_methods:
            return_has_type_hint = definition.get('returns') is not None
            if not return_has_type_hint:
                missing_type_hints.append('return')
        else:
            # Special methods are considered to have appropriate return type (None)
            return_has_type_hint = True
    
    return params_with_type_hints, missing_type_hints, return_has_type_hint

In [ ]:
#| export
def check_params_documentation(
    definition: Dict[str, Any],  # Definition dict from scanner
    source: str  # Function source code
) -> Tuple[Dict[str, bool], List[str], bool]:  # (params_documented, missing_params, return_documented)
    "Check parameter and return documentation for a function"
    params_documented = {}
    missing_params = []
    return_documented = True  # Default for classes
    
    # Only check for functions
    if definition['type'] in ['FunctionDef', 'AsyncFunctionDef']:
        # Try to use function object with fastcore.docments first
        func_obj = definition.get('func_obj')
        if func_obj and callable(func_obj):
            try:
                param_docs = extract_param_docs_from_func(func_obj)
            except Exception:
                # Fallback to source parsing
                param_docs = extract_param_docs(source)
        else:
            # Fallback to source parsing
            param_docs = extract_param_docs(source)
        
        # Check each parameter
        for arg in definition.get('args', []):
            param_name = arg['name']
            if param_name != 'self':  # Skip self parameter
                has_doc = param_name in param_docs
                params_documented[param_name] = has_doc
                if not has_doc:
                    missing_params.append(param_name)
        
        # Check return documentation if function has return annotation
        if definition.get('returns'):
            return_documented = check_return_doc(source)
            if not return_documented:
                missing_params.append('return')
    
    return params_documented, missing_params, return_documented

In [ ]:
#| export
def determine_compliance(
    has_docstring: bool,  # Whether definition has a docstring
    params_documented: Dict[str, bool],  # Which params have documentation
    return_documented: bool  # Whether return is documented
) -> bool:  # Overall compliance status
    "Determine if a definition is compliant based on documentation checks"
    return (
        has_docstring and 
        all(params_documented.values()) and 
        return_documented
    )

In [ ]:
#| export
def check_definition(
    definition: Dict[str, Any]  # Definition dict from scanner
) -> DocmentsCheckResult:  # Check result with compliance details
    "Check a function/class definition for docments compliance"
    name = definition['name']
    def_type = definition['type']
    source = definition['source']
    notebook = definition.get('notebook', 'unknown')
    func_obj = definition.get('func_obj')
    
    # Check for TODO placeholders only in documentation
    todo_count, has_todos = count_todos_in_docs(source, name)
    
    # Check for docstring - use function object if available
    if func_obj and callable(func_obj):
        try:
            has_docstring = check_has_docstring_from_func(func_obj)
        except Exception:
            has_docstring = check_has_docstring(source, name)
    else:
        has_docstring = check_has_docstring(source, name)
    
    # Check parameter and return documentation
    params_documented, missing_params, return_documented = check_params_documentation(definition, source)
    
    # Check type hints
    params_with_type_hints, missing_type_hints, return_has_type_hint = check_type_hints(definition)
    
    # Determine overall compliance
    is_compliant = determine_compliance(has_docstring, params_documented, return_documented)
    
    return DocmentsCheckResult(
        name=name,
        type=def_type,
        notebook=notebook,
        has_docstring=has_docstring,
        params_documented=params_documented,
        return_documented=return_documented,
        missing_params=missing_params,
        is_compliant=is_compliant,
        source=source,
        has_todos=has_todos,
        todo_count=todo_count,
        params_with_type_hints=params_with_type_hints,
        return_has_type_hint=return_has_type_hint,
        params_missing_type_hints=missing_type_hints
    )

In [ ]:
#| export
def check_notebook(
    nb_path: str  # Path to notebook file  
) -> None:  # Prints compliance report
    "Check a specific notebook for docments compliance"
    from pathlib import Path
    from cjm_nbdev_docments.scanner import scan_notebook
    from cjm_nbdev_docments.report import generate_text_report
    
    nb_path = Path(nb_path)
    definitions = scan_notebook(nb_path)
    results = [check_definition(defn) for defn in definitions]
    
    print(f"Checking {nb_path.name}:")
    print(generate_text_report(results, verbose=True))

In [ ]:
#| export
def check_function(
    func:Callable          # Function object to check
) -> DocmentsCheckResult:  # Check result for the function
    "Check a single function for docments compliance"
    import inspect
    
    # Get function source
    source = inspect.getsource(func)
    
    # Create a definition dict compatible with check_definition
    definition = {
        'name': func.__name__,
        'type': 'FunctionDef',
        'source': source,
        'notebook': 'runtime',
        'args': []
    }
    
    # Extract arguments
    sig = inspect.signature(func)
    for param_name, param in sig.parameters.items():
        if param_name != 'self':
            definition['args'].append({
                'name': param_name,
                'annotation': str(param.annotation) if param.annotation != param.empty else None
            })
    
    # Check for return annotation
    if sig.return_annotation != sig.empty:
        definition['returns'] = str(sig.return_annotation)
    
    result = check_definition(definition)
    
    # Print a simple report
    if result.is_compliant:
        print(f"✅ {result.name} is compliant")
    else:
        print(f"❌ {result.name} is not compliant")
        if not result.has_docstring:
            print("   - Missing docstring")
        if result.missing_params:
            print(f"   - Missing docs for: {', '.join(result.missing_params)}")
        if result.params_missing_type_hints:
            missing_type_hints = [p for p in result.params_missing_type_hints if p != 'return']
            if missing_type_hints:
                print(f"   - Missing type hints for: {', '.join(missing_type_hints)}")
            if 'return' in result.params_missing_type_hints:
                print("   - Missing return type hint")
    
    return result

In [ ]:
# Test checking a specific notebook
check_notebook("00_core.ipynb")

Checking 00_core.ipynb:
📚 Docments Compliance Report
Total definitions: 14
✅ Compliant: 14
❌ Non-compliant: 0


✅ Compliant definitions:
------------------------------

📓 00_core.ipynb:
  ✅ DocmentsCheckResult
  ✅ __post_init__
  ✅ extract_param_docs_from_func
  ✅ extract_param_docs
  ✅ check_return_doc
  ✅ count_todos_in_docs
  ✅ check_has_docstring_from_func
  ✅ check_has_docstring
  ✅ check_type_hints
  ✅ check_params_documentation
  ✅ determine_compliance
  ✅ check_definition
  ✅ check_notebook
  ✅ check_function


In [ ]:
# Test checking a function directly
def test_func(
    x: int,  # First number
    y: int = 0  # Second number  
) -> int:  # Sum of x and y
    "Add two numbers"
    return x + y

check_function(test_func)

✅ test_func is compliant


DocmentsCheckResult(name='test_func', type='FunctionDef', notebook='runtime', has_docstring=True, params_documented={'x': True, 'y': True}, return_documented=True, missing_params=[], is_compliant=True, source='def test_func(\n    x: int,  # First number\n    y: int = 0  # Second number  \n) -> int:  # Sum of x and y\n    "Add two numbers"\n    return x + y\n', has_todos=False, todo_count=0, params_with_type_hints={'x': True, 'y': True}, return_has_type_hint=True, params_missing_type_hints=[])

In [ ]:
# Test with a non-compliant function
def bad_func(x, y):
    return x + y

check_function(bad_func)

❌ bad_func is not compliant
   - Missing docstring
   - Missing docs for: x, y
   - Missing type hints for: x, y
   - Missing return type hint


DocmentsCheckResult(name='bad_func', type='FunctionDef', notebook='runtime', has_docstring=False, params_documented={'x': False, 'y': False}, return_documented=True, missing_params=['x', 'y'], is_compliant=False, source='def bad_func(x, y):\n    return x + y\n', has_todos=False, todo_count=0, params_with_type_hints={'x': False, 'y': False}, return_has_type_hint=False, params_missing_type_hints=['x', 'y', 'return'])

In [ ]:
# Test refined TODO detection
test_cases = [
    # Case 1: TODOs only in documentation (should count)
    '''def func_with_doc_todos(
    x: int,  # TODO: Add description
    y: str  # TODO: Add description  
) -> bool:  # TODO: Add return description
    "TODO: Add function description"
    return True''',
    
    # Case 2: TODOs in function body but not docs (should NOT count)
    '''def func_with_code_todos(
    x: int,  # Parameter x
    y: str  # Parameter y
) -> bool:  # Returns boolean
    "Function with proper docs"
    # TODO: Implement this feature later
    todo_list = ["item1", "item2"]  # Variable named todo
    return True''',
    
    # Case 3: Mixed TODOs (should only count doc ones)
    '''def mixed_todos(
    x: int  # TODO: Add description
) -> bool:
    "Proper function description"
    # TODO: Refactor this code
    todo_items = []
    return True'''
]

for i, source in enumerate(test_cases, 1):
    test_def = {
        'name': f'test_func_{i}',
        'type': 'FunctionDef',
        'source': source,
        'notebook': 'test.ipynb',
        'args': [{'name': 'x', 'annotation': 'int'}],
        'returns': 'bool'
    }
    
    result = check_definition(test_def)
    print(f"Case {i}: {result.name}")
    print(f"  TODO count: {result.todo_count}")
    print(f"  Has TODOs: {result.has_todos}")
    print(f"  Compliant: {result.is_compliant}")
    print()

Case 1: test_func_1
  TODO count: 3
  Has TODOs: True
  Compliant: False

Case 2: test_func_2
  TODO count: 0
  Has TODOs: False
  Compliant: False

Case 3: test_func_3
  TODO count: 1
  Has TODOs: True
  Compliant: False



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()